In [1]:
def makeREADMEitr(iteration,dtint):
    
    rmifile = './orbits/'+str(nsample*nbatch)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(simname)+'_'+str(iteration)+'_README.txt'
    rm = open(rmifile,'w+')
    
    rm.write('The pattern speed at the beginning and end of this iteration is Omega_i = '+str(np.round(omegat[nphio[iteration]],4))+' and Omega_f = '+str(np.round(omegat[nphio[iteration+1]],4))+' in natural units.\n')
    rm.write('\n')

    rm.write('The time index marking the beginning and end of this iteration is nphio_i = '+str(nphio[iteration])+' and Omega_f = '+str(nphio[iteration+1])+' in natural units.\n')
    rm.write('\n')

    rm.write('### Output configurations\n')
    rm.write('Shape of final orbits array in cylindrical coordinates is '+str(np.shape(orp))+'\n')
    rm.write('   There are '+str(np.shape(orp[:,0,0])[0])+' coordinates [R,phi,z,vR,vphi,vz]\n')
    rm.write('   There are '+str(np.shape(orp[0,:,0])[0])+' orbits\n')
    rm.write('   There are '+str(np.shape(orp[0,0,:])[0])+' timestep outputs for each\n')

    rm.write('Shape of final orbits array in cartesian coordinates is '+str(np.shape(oxy))+'\n')
    rm.write('   There are '+str(np.shape(oxy[:,0,0])[0])+' coordinates [x,y,z,vx,vy,vz]\n')
    rm.write('   There are '+str(np.shape(oxy[0,:,0])[0])+' orbits\n')
    rm.write('   There are '+str(np.shape(oxy[0,0,:])[0])+' timestep outputs for each\n')

    rm.write('Shape of final actions array is '+str(np.shape(oa))+'\n')
    rm.write('   There are '+str(np.shape(oa[:,0])[0])+' coordinates [jR_f,jphi_f,jz_f,jR_o,jphi_o,jz_o]\n')
    rm.write('   There are '+str(np.shape(oa[0,:])[0])+' orbits\n')

    rm.write('\n')
    rm.write('Created at: '+str(datetime.now())+'\n')
    rm.write('Time lapsed to integrate and save: '+str(datetime.now()-dtint))

    rm.close()

In [ ]:
### Integrate from last step of grow bar as defined by startbatch and from startsimname
### Save it and write README

dt00 = datetime.now()
print('\033[1;34mStart time is: ',dt00)
print('\n')

nphio = maxpoints[0]
nphio = np.insert(nphio,0,0)
#print('phit is ',phit[nphio])
#print('shape of phit is ',np.shape(nphio))
nitr = len(nphio)-1

for iteration in range(21,nitr):  ###### UPDATE each time.  Crashes after two iterations. #####

    # Start clock
    from datetime import datetime
    dt0 = datetime.now()
    print('\033[1;34mSetting up for integration '+str(iteration+1)+' of '+str(nitr)+'. Iteration #'+str(iteration))
    print('   at time is: ',dt0)
    
    # Define parameter space

    if iteration < nitr+1:
        ts= tvector[nphio[iteration]:nphio[iteration+1]]

    # Set up orbits
    dt1 = datetime.now()
    print('\n')
    print('\033[0;30m   Setting up orbits')
    o = orbitsetup(nbatch,nsample,iteration)
    print('   Time lapsed to set up orbits = ', datetime.now()-dt0)

    # Integrate orbits
    dt2 = datetime.now()
    print('\n')
    print('   Integrating '+str(nsample*nbatch)+' orbits')
    pot = [mwp,btp]
    o.integrate(ts,pot,method='leapfrog')
    print('   Time lapsed to intigrate orbits = ', datetime.now()-dt2)

    # Save orbits
    dt3 = datetime.now()
    print('\n')
    print('   Saving orbits using cylindrical coordinates')
    outnamerp = './orbits/'+str(simname)+'/'+str(nsample*nbatch)+'N_qiDF_'+dpType+'p_'+AAType+'_orbits_cyl_'+str(simname)+'_'+str(iteration)+'.npy'
    orp = np.array([o.R(ts[[0,-1]]),o.phi(ts[[0,-1]]),o.z(ts[[0,-1]]),o.vR(ts[[0,-1]]),o.vT(ts[[0,-1]]),o.vz(ts[[0,-1]])])
    np.save(outnamerp,orp)
    print('   Shape of final orbits array is '+str(np.shape(orp)))
    print('   There are '+str(np.shape(orp[:,0,0])[0])+' coordinates [R,phi,z,vR,vphi,vz]')
    print('   There are '+str(np.shape(orp[0,:,0])[0])+' orbits')
    print('   There are '+str(np.shape(orp[0,0,:])[0])+' timestep outputs for each')
    print('   Time lapsed to save orbits = ', datetime.now()-dt3)
    print('\n')

    print('   Saving orbits using cartesian coordinates')
    outnamexy = './orbits/'+str(simname)+'/'+str(nsample*nbatch)+'N_qiDF_'+dpType+'p_'+AAType+'_orbits_cart_'+str(simname)+'_'+str(iteration)+'.npy'
    oxy = np.array([o.x(ts[[0,-1]]),o.y(ts[[0,-1]]),o.z(ts[[0,-1]]),o.vx(ts[[0,-1]]),o.vy(ts[[0,-1]]),o.vz(ts[[0,-1]])])
    np.save(outnamexy,oxy)
    print('   Shape of final orbits array is '+str(np.shape(oxy)))
    print('   There are '+str(np.shape(oxy[:,0,0])[0])+' coordinates [x,y,z,vx,vy,vz]')
    print('   There are '+str(np.shape(oxy[0,:,0])[0])+' orbits')
    print('   There are '+str(np.shape(oxy[0,0,:])[0])+' timestep outputs for each')
    print('   Time lapsed to save orbits = ', datetime.now()-dt3)
    dt4 = datetime.now()
    print('\n')
    print('   Saving final actions')
    outnamea = './orbits/'+str(simname)+'/'+str(nsample*nbatch)+'N_qiDF_'+dpType+'p_'+AAType+'_actions_'+str(simname)+'_'+str(iteration)+'.npy'
    oai = np.array([o(ts[0]).jr(mwp),o(ts[0]).jp(mwp),o(ts[0]).jz(mwp)])
    oaf = np.array([o(ts[-1]).jr(mwp),o(ts[-1]).jp(mwp),o(ts[-1]).jz(mwp)])
    oa = np.append(oaf,oai,axis=0)
    np.save(outnamea,oa)
    print('   Shape of final actions array is '+str(np.shape(oa)))
    print('   There are '+str(np.shape(oa[:,0])[0])+' coordinates [jR_f,jphi_f,jz_f,jR_o,jphi_o,jz_o]')
    print('   There are '+str(np.shape(oa[0,:])[0])+' orbits')
    print('   Time lapsed to save actions = ', datetime.now()-dt4)

    # Save README file
    print('\n')
    print('   Making README file and preparing for next iteration')
    makeREADMEitr(iteration, dt0)
    o = None
    print('\n')

    print('   Total time lapsed to integrate and save = ', datetime.now()-dt0)

print('   TOTAL total time lapsed = ', datetime.now()-dt00)




In [ ]:
# Setup meta info in numpy arrays
nphio = maxpoints[0]
nphio = np.insert(nphio,0,0)

tvectorfile = './orbits/'+str(nsample*nbatch)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(simname)+'_tvector.npy'
omegatfile =  './orbits/'+str(nsample*nbatch)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(simname)+'_Omegat.npy'
nphiofile =  './orbits/'+str(nsample*nbatch)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(simname)+'_nphio.npy'

Otsave = omegat[nphio]
np.save(tvectorfile,tvector)
np.save(omegatfile,Otsave)
np.save(nphiofile,nphio)